In [1]:
from MyLib import load_celebA
from MyLib import VAE
import torch
from torch import optim
import torch.nn as nn
from torch.nn import functional as F
from torchvision.utils import save_image
from torch.autograd import Variable
import numpy as np
import pdb
import os

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
    if classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 0.02)
        
        
def loss_function(recon_x,x,mu,logvar):
    BCE = F.binary_cross_entropy(recon_x,x.view(-1,4096))
    KLD = -.5 * torch.sum(1+logvar-mu.pow(2)-logvar.exp())
    KLD /= batch_size*4096
    #pdb.set_trace()
    KLD = KLD 
    return BCE+KLD, BCE, KLD


mse = nn.MSELoss()
def fpl_criterion(recon_features, targets):
    fpl = 0
    for f, target in zip(recon_features, targets):
        fpl += mse(f, target.detach())#.div(f.size(1))
    return fpl

bce_loss = nn.MSELoss()

In [16]:
def train(epoch):
    
    cell_train_loss = 0    
    linear_model.train()
    '''
    for batch_idx, data in enumerate(cell_train):
        
        #load data in the dataloader
        image = Variable(data['image']).float().view(-1,1,64,64).cuda()
        cell_acti = Variable(data['cell']).float().cuda()
        
        #train the model 
        optimizer.zero_grad()
        recon_batch, mu, logvar,z = model(image)
        Linear_cell = linear_model(z)
        #pdb.set_trace()
        
        image_3d = image.expand(-1,3,-1,-1)
        recon_batch_3d = recon_batch.view(-1,1,64,64).expand(-1,3,-1,-1)
        loss = bce_loss(Linear_cell,cell_acti)
        loss.backward()
        cell_train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % 10 == 0:
            #pdb.set_trace()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(cell_train.dataset),
                100. * batch_idx / len(cell_train),
                loss.data[0] / len(data)
            ))
    
    '''
    model.train()
    #Linear_model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        #data = data.view(-1,1,64,64)
        data = Variable(data['image']).float().view(-1,1,64,64).cuda()
        
        #image_3d = image.expand(-1,3,-1,-1)
        optimizer.zero_grad()
        recon_batch, mu, logvar,z = model(data)
        
        
        
        data_3d = data.expand(-1,3,-1,-1)
        #pdb.set_trace()
        vgg_feature = descriptor(data_3d)
        
        recon_batch = recon_batch.view(-1,1,64,64)
        recon_batch_3d = recon_batch.expand(-1,3,-1,-1)
        recon_vgg_feature = descriptor(recon_batch_3d)

        fpl = fpl_criterion(recon_vgg_feature, vgg_feature)
        
        loss,bce,kld = loss_function(recon_batch, data, mu, logvar)
        #pdb.set_trace()
        #loss.backward()
        
            
        kld.backward(retain_graph=True)
        fpl.backward()
        loss = kld+fpl
        #bce.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % 10 == 0:
            #pdb.set_trace()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0] / len(data)
            ))
            print ('kld=',kld.data[0]/len(data))
            
        '''
        if batch_idx % 40000 == 0:    
            n = min(data.size(0), 8)
            comparison = torch.cat([data[:n],recon_batch.view(args.batch_size, 1, 64, 64)[:n]])
            save_image(comparison.data.cpu(),'results/reconstruction_'+'beta_'+str(batch_idx)+'perfomance.png')
        '''

        
    #pdb.set_trace()   
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / (len(train_loader)*batch_size)))
    
    #print('====> Epoch: {} Average cell loss: {:.4f}'.format(
            #epoch, cell_train_loss / (len(cell_train)*batch_size)))

def test(epoch):
    model.eval()
    test_loss = 0
    for i, data in enumerate(test_loader):
        #data = data.view(-1,1,64,64)
        
        data = Variable(data['image'], volatile=True).float().view(-1,1,64,64)
        #pdb.set_trace()
        data = data.cuda()
        data_3d = data.expand(-1,3,-1,-1)
        data_3d = data_3d.cuda()
        recon_batch, mu, logvar,z = model(data)
        recon_batch_3d = recon_batch.view(-1,1,64,64)
        recon_batch_3d = recon_batch_3d.expand(-1,3,-1,-1)
        
        test_lo,bce,kld = loss_function(recon_batch, data, mu, logvar)
        
        vgg_feature = descriptor(data_3d)
        recon_vgg_feature = descriptor(recon_batch_3d)
        fpl = fpl_criterion(recon_vgg_feature, vgg_feature)
        test_lo = kld+fpl
        
        
        test_loss += test_lo
        #pdb.set_trace()
        
        if i == 0:
          
            n = min(data.size(0), 8)
          
            comparison = torch.cat([data[:n],
                                  recon_batch.view(batch_size, 1, 64, 64)[:n]])
            save_image(comparison.data.cpu(),
                     'results/reconstruction_' + str(epoch)  +'.png', nrow=n)
        #print ('processed batch '+str(i))
        
        '''
        linear_model.eval()
        cell_test_loss = 0
        for batch_idx, data in enumerate(cell_test):
            image = Variable(data['image']).float().view(-1,1,64,64).cuda()
            image_3d = image.expand(-1,3,-1,-1)
            
            cell_acti = Variable(data['cell']).float().cuda()
            #optimizer.zero_grad()
            recon_batch, mu, logvar,z = model(image)
            recon_batch_3d = recon_batch.expand(-1,3,-1,-1)
            
            Linear_cell = linear_model(z)
            loss = bce_loss(Linear_cell,cell_acti)
            #loss.backward()
            cell_test_loss += loss.data[0]
            #optimizer.step()
        '''
            
        '''
            if batch_idx % 10 == 0:
            #pdb.set_trace()
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(cell_train.dataset),
                    100. * batch_idx / len(cell_train),
                    loss.data[0] / len(data)
                )) 
        '''   
    test_loss /= len(test_loader)*batch_size
    #cell_test_loss /= len(cell_test)*batch_size
    print('====> Test set loss =', test_loss)
    #print('====> Cell Test set loss =', cell_test_loss)

In [4]:

batch_size = 128

_,train_loader,test_loader = load_celebA.celeba_dataset()

cell_loader,cell_train, cell_test = load_celebA.facial_dataset()
model = VAE.DC_VAE()

linear_model = VAE.Linear_model()

model.apply(weights_init)
linear_model.apply(weights_init)
model.cuda()
linear_model.cuda()

descriptor = VAE.pre_trained_vgg()
descriptor.cuda()

params = list(linear_model.parameters()) + list(model.parameters())
optimizer = optim.Adam(params, lr=2e-4)

In [18]:
for epoch in range(1, 101):
    train(epoch)
    test(epoch)
    sample = Variable(torch.randn(64,32))
    
    sample = sample.cuda()
    sample = model.decoder(sample).cpu()
    save_image(sample.data.view(64, 1, 64, 64),
               'recon_result/sample_' + str(epoch) +'beta' + '.png')

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([128, 4096])) that is different to the input size (torch.Size([128, 1, 64, 64])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Train Epoch: 1 [0/202599 (0%)]	Loss: 0.053400
kld= 0.0005124068120494485
Train Epoch: 1 [1280/202599 (1%)]	Loss: 0.050140
kld= 0.0005144855240359902
Train Epoch: 1 [2560/202599 (2%)]	Loss: 0.052932
kld= 0.0005420737434178591
Train Epoch: 1 [3840/202599 (2%)]	Loss: 0.052311
kld= 0.0005312341381795704
Train Epoch: 1 [5120/202599 (3%)]	Loss: 0.053077
kld= 0.0005342192016541958
Train Epoch: 1 [6400/202599 (4%)]	Loss: 0.050819
kld= 0.0005394243635237217
Train Epoch: 1 [7680/202599 (5%)]	Loss: 0.050120
kld= 0.0005567402113229036
Train Epoch: 1 [8960/202599 (6%)]	Loss: 0.052675
kld= 0.0005356252659112215
Train Epoch: 1 [10240/202599 (6%)]	Loss: 0.051419
kld= 0.0005367777775973082
Train Epoch: 1 [11520/202599 (7%)]	Loss: 0.052418
kld= 0.0005418527871370316
Train Epoch: 1 [12800/202599 (8%)]	Loss: 0.052778
kld= 0.0005421493551693857
Train Epoch: 1 [14080/202599 (9%)]	Loss: 0.050039
kld= 0.0005501635605469346
Train Epoch: 1 [15360/202599 (9%)]	Loss: 0.053303
kld= 0.0005073416396044195
Train Epoc

Train Epoch: 1 [145920/202599 (90%)]	Loss: 0.048350
kld= 0.000587868969887495
Train Epoch: 1 [147200/202599 (91%)]	Loss: 0.049484
kld= 0.0005792881711386144
Train Epoch: 1 [148480/202599 (92%)]	Loss: 0.050172
kld= 0.0006023840396665037
Train Epoch: 1 [149760/202599 (92%)]	Loss: 0.047709
kld= 0.0006025497568771243
Train Epoch: 1 [151040/202599 (93%)]	Loss: 0.051257
kld= 0.0005875317147001624
Train Epoch: 1 [152320/202599 (94%)]	Loss: 0.049681
kld= 0.0005881216493435204
Train Epoch: 1 [153600/202599 (95%)]	Loss: 0.046543
kld= 0.0005988692864775658
Train Epoch: 1 [154880/202599 (96%)]	Loss: 0.047430
kld= 0.0006011930527165532
Train Epoch: 1 [156160/202599 (96%)]	Loss: 0.048901
kld= 0.0005951320636086166
Train Epoch: 1 [157440/202599 (97%)]	Loss: 0.047957
kld= 0.0005942370044067502
Train Epoch: 1 [158720/202599 (98%)]	Loss: 0.049718
kld= 0.0005961955757811666
Train Epoch: 1 [160000/202599 (99%)]	Loss: 0.048962
kld= 0.0006134581053629518
Train Epoch: 1 [161280/202599 (99%)]	Loss: 0.048961
k

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([31, 4096])) that is different to the input size (torch.Size([31, 1, 64, 64])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


====> Epoch: 1 Average loss: 0.0502
====> Test set loss = Variable containing:
1.00000e-02 *
  4.8915
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Train Epoch: 2 [0/202599 (0%)]	Loss: 0.049168
kld= 0.0005872749025002122
Train Epoch: 2 [1280/202599 (1%)]	Loss: 0.049911
kld= 0.0005912618944421411
Train Epoch: 2 [2560/202599 (2%)]	Loss: 0.047308
kld= 0.0005982076982036233
Train Epoch: 2 [3840/202599 (2%)]	Loss: 0.049470
kld= 0.0006072113174013793
Train Epoch: 2 [5120/202599 (3%)]	Loss: 0.049260
kld= 0.0006096785655245185
Train Epoch: 2 [6400/202599 (4%)]	Loss: 0.047694
kld= 0.0006168883410282433
Train Epoch: 2 [7680/202599 (5%)]	Loss: 0.047376
kld= 0.0006147645763121545
Train Epoch: 2 [8960/202599 (6%)]	Loss: 0.048666
kld= 0.0005991743528284132
Train Epoch: 2 [10240/202599 (6%)]	Loss: 0.051221
kld= 0.0005967324832454324
Train Epoch: 2 [11520/202599 (7%)]	Loss: 0.048221
kld= 0.0005966068129055202
Train Epoch: 2 [12800/202599 (8%)]	Loss: 0.049488
kld= 0.0006017843261361122
Train Epoch: 2 [14

...

Train Epoch: 92 [131840/202599 (81%)]	Loss: 0.035275
kld= 0.0008819966460578144
Train Epoch: 92 [133120/202599 (82%)]	Loss: 0.037007
kld= 0.0008706302032805979
Train Epoch: 92 [134400/202599 (83%)]	Loss: 0.036539
kld= 0.0008855146006681025
Train Epoch: 92 [135680/202599 (84%)]	Loss: 0.037633
kld= 0.0008788011036813259
Train Epoch: 92 [136960/202599 (84%)]	Loss: 0.037951
kld= 0.0008823635289445519
Train Epoch: 92 [138240/202599 (85%)]	Loss: 0.036470
kld= 0.0008850513258948922
Train Epoch: 92 [139520/202599 (86%)]	Loss: 0.040487
kld= 0.0008739641634747386
Train Epoch: 92 [140800/202599 (87%)]	Loss: 0.038704
kld= 0.0008793668821454048
Train Epoch: 92 [142080/202599 (88%)]	Loss: 0.038512
kld= 0.0008841828675940633
Train Epoch: 92 [143360/202599 (88%)]	Loss: 0.038165
kld= 0.0008835766930133104
Train Epoch: 92 [144640/202599 (89%)]	Loss: 0.039772
kld= 0.0008800733485259116
Train Epoch: 92 [145920/202599 (90%)]	Loss: 0.036873
kld= 0.0008724461658857763
Train Epoch: 92 [147200/202599 (91%)]	Lo

Train Epoch: 93 [99840/202599 (62%)]	Loss: 0.037116
kld= 0.0008790046558715403
Train Epoch: 93 [101120/202599 (62%)]	Loss: 0.037794
kld= 0.0008631901000626385
Train Epoch: 93 [102400/202599 (63%)]	Loss: 0.038693
kld= 0.0008689430542290211
Train Epoch: 93 [103680/202599 (64%)]	Loss: 0.038760
kld= 0.0008787389378994703
Train Epoch: 93 [104960/202599 (65%)]	Loss: 0.038719
kld= 0.0008821832598187029
Train Epoch: 93 [106240/202599 (66%)]	Loss: 0.037864
kld= 0.0008825415279716253
Train Epoch: 93 [107520/202599 (66%)]	Loss: 0.038947
kld= 0.0008672358235344291
Train Epoch: 93 [108800/202599 (67%)]	Loss: 0.038745
kld= 0.0008689763490110636
Train Epoch: 93 [110080/202599 (68%)]	Loss: 0.037754
kld= 0.0008955323719419539
Train Epoch: 93 [111360/202599 (69%)]	Loss: 0.035791
kld= 0.0008919417159631848
Train Epoch: 93 [112640/202599 (69%)]	Loss: 0.037537
kld= 0.0008831362356431782
Train Epoch: 93 [113920/202599 (70%)]	Loss: 0.038554
kld= 0.0008689125534147024
Train Epoch: 93 [115200/202599 (71%)]	Los

Train Epoch: 94 [67840/202599 (42%)]	Loss: 0.037332
kld= 0.0008802948868833482
Train Epoch: 94 [69120/202599 (43%)]	Loss: 0.036708
kld= 0.0008806512923911214
Train Epoch: 94 [70400/202599 (43%)]	Loss: 0.037943
kld= 0.0008735141018405557
Train Epoch: 94 [71680/202599 (44%)]	Loss: 0.036467
kld= 0.0008770213462412357
Train Epoch: 94 [72960/202599 (45%)]	Loss: 0.037930
kld= 0.0008730054832994938
Train Epoch: 94 [74240/202599 (46%)]	Loss: 0.037824
kld= 0.00086510757682845
Train Epoch: 94 [75520/202599 (47%)]	Loss: 0.035729
kld= 0.00089278380619362
Train Epoch: 94 [76800/202599 (47%)]	Loss: 0.037058
kld= 0.0008609267533756793
Train Epoch: 94 [78080/202599 (48%)]	Loss: 0.037835
kld= 0.0008802017546258867
Train Epoch: 94 [79360/202599 (49%)]	Loss: 0.036325
kld= 0.0008957528625614941
Train Epoch: 94 [80640/202599 (50%)]	Loss: 0.037123
kld= 0.0008862304384820163
Train Epoch: 94 [81920/202599 (51%)]	Loss: 0.037591
kld= 0.0008645083289593458
Train Epoch: 94 [83200/202599 (51%)]	Loss: 0.038543
kld=

Train Epoch: 95 [35840/202599 (22%)]	Loss: 0.038162
kld= 0.0008690347312949598
Train Epoch: 95 [37120/202599 (23%)]	Loss: 0.038208
kld= 0.0008838505600579083
Train Epoch: 95 [38400/202599 (24%)]	Loss: 0.036887
kld= 0.0008802819647826254
Train Epoch: 95 [39680/202599 (24%)]	Loss: 0.036547
kld= 0.0008970275521278381
Train Epoch: 95 [40960/202599 (25%)]	Loss: 0.039069
kld= 0.0008787109982222319
Train Epoch: 95 [42240/202599 (26%)]	Loss: 0.036358
kld= 0.0008870890596881509
Train Epoch: 95 [43520/202599 (27%)]	Loss: 0.037865
kld= 0.0008858592482283711
Train Epoch: 95 [44800/202599 (28%)]	Loss: 0.037491
kld= 0.0008915790822356939
Train Epoch: 95 [46080/202599 (28%)]	Loss: 0.037129
kld= 0.0008751977584324777
Train Epoch: 95 [47360/202599 (29%)]	Loss: 0.035967
kld= 0.0008938860846683383
Train Epoch: 95 [48640/202599 (30%)]	Loss: 0.037545
kld= 0.000884527456946671
Train Epoch: 95 [49920/202599 (31%)]	Loss: 0.034945
kld= 0.0008845657575875521
Train Epoch: 95 [51200/202599 (32%)]	Loss: 0.037674
k

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
model_test = VAE.DC_VAE()
model_test = model_test.cuda()
model_test.load_state_dict(torch.load('DC_VAE100epoch.pt'))

In [10]:
sample = Variable(torch.randn(64,32))
sample = sample.cuda()
model_test = model_test.cuda()
sample = model_test.decoder(sample).cpu()
save_image(sample.data.view(64,1,64,64),'recon_result/samp__qwq_1.png')

In [ ]:


for epoch in range(1,101):
    model_test.train()
    optimizer.zero_grad()
    for batch_idx,data in enumerate(cell_loader):
    #model_test.train()
        image = Variable(data['image']).float().view(-1,1,64,64).cuda()
        recon_batch,mu,logvar,z = model_test(image)
        recon_batch = recon_batch.view(-1,1,64,64)
        loss,bce,kld = loss_function(recon_batch, image, mu, logvar)
        loss.backward()
    
        optimizer.step()
        recon_batch = recon_batch.cpu()
        image = image.cpu()
        if epoch%10 == 0:
            save_image(recon_batch.data.view(-1,1,64,64),
                      'Re_sults/'+'recon'+str(batch_idx)+'epoch'+str(epoch)+'.png')
            save_image(image.data.view(-1,1,64,64),
                      'Re_sults/'+str(batch_idx)+'epoch'+str(epoch)+'.png')
            arr = z.data.cpu().numpy()
            np.savetxt('Re_sults/' + str(batch_idx)+'epoch'+str(epoch)+'.csv',arr)
    #print ('epoch'+str(epoch))
    

In [20]:
torch.save(model.state_dict(),'DFC_VAE_100epoch.pt')

In [1]:
a = os.listdir('/home/ubuntu/wkhan/VAE/dataset/processed_img')
b = a.sort(key= lambda x: float(x.strip('.png')))


NameError: name 'os' is not defined